# Ethereum MainNet 블록을 Big-Query로 탐색하기

----

* **DATE** : 2021.11.12
* **AUTHOR** : Kang Sang Jae
* **Reference** : 

    * [Full relational diagram for Ethereum public data on Google BigQuery](https://medium.com/google-cloud/full-relational-diagram-for-ethereum-public-data-on-google-bigquery-2825fdf0fb0b)
    * [SQL On Ethereum: How To Work With All the Data From a Transaction](https://towardsdatascience.com/sql-on-ethereum-how-to-work-with-all-the-data-from-a-transaction-103f94f902e5)
    * [What is crypto data](https://towardsdatascience.com/what-is-crypto-data-fef70ecadf7b)
    * [Unlocking the Secrets of an Ethereum Transaction](https://medium.com/coinmonks/discovering-the-secrets-of-an-ethereum-transaction-64febb00935c)
    * [Decoding Ethereum smart contract data](https://towardsdatascience.com/decoding-ethereum-smart-contract-data-eed513a65f76)
    * [The data behind the NFT hype](https://towardsdatascience.com/the-data-behind-the-nft-hype-32f6f92c27cb)
    * [Understanding event logs on the Ethereum blockchain](https://medium.com/mycrypto/understanding-event-logs-on-the-ethereum-blockchain-f4ae7ba50378)

In [1]:
%matplotlib inline
import os
import json
import requests
from functools import lru_cache
from web3 import Web3
import pandas as pd
import matplotlib.pyplot as plt
import eth_utils

# GCP 인증서 다운 받기
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = (
    os.path.abspath("../keys/gcp.json"))

# Infura Key file
KEY_DICT = json.load(open("../keys/infura.json",'r'))

project_id = KEY_DICT['projectId']

ETHERSCAN_DICT = json.load(open("../keys/etherscan.json"))

In [2]:
@lru_cache(maxsize=1024)
def get_abi(contract_address):
    apiKey = ETHERSCAN_DICT['apiKey']
    return requests.get(f"https://api.etherscan.io/api?apiKey={apiKey}&module=contract&action=getabi&address={contract_address}").json()

In [3]:
%load_ext google.cloud.bigquery

## 1.  SWAP의 대상이 되는 토큰 살펴보기

**1.1 현재 시장 내에 있는 `ERC-20` Token 리스트를 가져오기**

````
Token data. Data is exported using https://github.com/medvedev1088/ethereum-etl
````


| Field name | Type | Mode | Description |
| --- | --- | --- | --- |
| address | STRING | REQUIRED | The address of the ERC20 token | 
| symbol | STRING | NULLABLE | The symbol of the ERC20 token | 
| name | STRING | NULLABLE | The name of the ERC20 token | 
| decimals | STRING | NULLABLE | The number of decimals the token uses. | Use safe_cast for casting to NUMERIC or FLOAT64 | 
| total_supply | STRING | NULLABLE | The total token supply. Use safe_cast for casting to NUMERIC or FLOAT64 | 
| block_timestamp | TIMESTAMP | REQUIRED | Timestamp of the block where | this token was created | 
| block_number | INTEGER | REQUIRED | Block number where this token was created | 
| block_hash | STRING | REQUIRED | Hash of the block where this token was created

In [4]:
%%bigquery token_df

SELECT * 
FROM `bigquery-public-data.crypto_ethereum.tokens`;

Downloading: 100%|██████████| 207353/207353 [00:10<00:00, 19537.05rows/s]


In [5]:
token_df

,address,symbol,name,decimals,total_supply,block_timestamp,block_number,block_hash
0,0x54994ba4b4a42297b3b88e27185cde1f51dca288,Alia,Xanalia,None,0,2021-11-12 07:02:07+00:00,13599860,0x35e88b97eb3a4091fa850e273b127eb444186a3a716b...
1,0xf2693018f665b3d30cafacb1d5407b997c9ce432,MAMI,Malaysian Asset Marry Investment,18,0,2021-11-12 09:53:56+00:00,13600633,0x01a2c7cf35f4cb134c2023e927d0465fa662f0adf919...
2,0xfc1d7291ce8e8e081fba1731850f7ece65cf7bfe,GAN,GanCoin,4,1000000000000,2021-11-12 03:06:23+00:00,13598811,0x878c88b4bd19d26e6768353ce9ae580afec26b67a7a4...
3,0x4907cda3d215d363e3a6d731673c9eb54d6c9ccc,KOR,KOREA Token,8,1000000000000,2021-11-12 02:30:24+00:00,13598652,0x6920da0165c722775133351180f03dfb8a8045d5e432...
4,0x80647538b718b62001b1edb12cf3935d4ccf867f,YAGMI,YAGMI,18,100000000000000000000000000,2021-11-12 00:23:41+00:00,13598082,0xe8603b0b4dc96d01903bb28056b146f52cc5cc8d8b0f...
...,...,...,...,...,...,...,...,...
207348,0x5bbfbc195bb980286d5a3a9c5e57070379a55d9f,MoM,Money of Money,238,50000000,2017-08-26 13:41:06+00:00,4206062,0x2f369daee14fd74964ad51ecdc5a2dfda05fcbefd95b...
207349,0x3f58b1d0e6f9a9efb021afb9bde847d2d39bdb9d,BTS,BeatsToken,238,390000000,2019-12-20 18:15:32+00:00,9136812,0xf1ad8b671d1e4f223422f465336595442ff956ea8937...
207350,0x6857910f1b57600f1a8a9b4dc468f6937a4e180e,DANK,DankCoin,254,1157920892373161954235709850086879078532699846...,2017-08-06 03:53:02+00:00,4122509,0x2482ee5c116f41ea6d193b3fdb74ac6696712338739e...
207351,0x4b258bf860db33ed1201181529453afd67d41d61,CAPY,CapyCOIN,255,1157920892373161954235709850086879078532699846...,2017-05-09 07:47:10+00:00,3676169,0x29c1d34cd0c1833027c07dd49e37b79df5d1ed2226a3...


**1.2 최근 1달 간 시장 내 `ERC-20` Token의 거래량을 가져오기**

````
The most popular type of transaction on the Ethereum blockchain invokes a contract of type ERC20 to perform a transfer operation, moving some number of tokens from one 20-byte address to another 20-byte address. This table contains the subset of those transactions and has further processed and denormalized the data to make it easier to consume for analysis of token transfer events. Data is exported using https://github.com/medvedev1088/ethereum-etl
````


| Field name | Type | Mode | Description |
| --- | --- | --- | --- |
| token_address | STRING | REQUIRED | ERC20 token address | 
| from_address | STRING | NULLABLE | Address of the sender | 
| to_address | STRING | NULLABLE | Address of the receiver | 
| value | STRING | NULLABLE | Amount of tokens transferred (ERC20) / id of the token transferred (ERC721). Use safe_cast for casting to NUMERIC or FLOAT64 | 
| transaction_hash | STRING | REQUIRED | Transaction hash | 
| log_index | INTEGER | REQUIRED | Log index in the transaction receipt | 
| block_timestamp | TIMESTAMP | REQUIRED | Timestamp of the block where this transfer was in | 
| block_number | INTEGER | REQUIRED | Block number where this transfer was in | 
| block_hash | STRING | REQUIRED | Hash of the block where this transfer was in

In [6]:
%%bigquery token_transfer_df

SELECT 
    token_address as address, 
    COUNT(*) as totalCount, 
    SUM(SAFE_CAST(value AS FLOAT64)) as totalAmount

FROM 
   `bigquery-public-data.crypto_ethereum.token_transfers`

WHERE 1=1
    AND DATE(block_timestamp) >= '2021-10-01' 
    AND DATE(block_timestamp) < '2021-11-01'
    AND SAFE_CAST(value AS FLOAT64) IS NOT NULL
GROUP BY token_address

Downloading: 100%|██████████| 32106/32106 [00:04<00:00, 7519.91rows/s]


**거래량(거래금액) 상위 10개**

In [7]:
top10_address_by_amount = (
    token_transfer_df
    [token_transfer_df.address.isin(token_df.address)]
    .sort_values("totalAmount",ascending=False)
    .iloc[:10]
    .address
)

In [8]:
token_df[token_df.address.isin(top10_address_by_amount)]

,address,symbol,name,decimals,total_supply,block_timestamp,block_number,block_hash
8047,0x095c3f21a8ddf9307e5d7091f8ee5d1dcfd2d705,vatomlabs.avocados::1182::v1,vatomlabs.avocados::1182::v1::avocadosfrommexi...,None,1,2020-01-23 18:17:08+00:00,9339532,0x5ac454f25eee5041c2712d7458c7bd435fcc5a3426b7...
10004,0x970fbc3126e19dc57f6649133436c2def00051a8,WAW,EterArt,None,12,2018-12-12 17:38:37+00:00,6874244,0xb1ed64956901ab7fc82a4b891fa65e667866f60500c9...
10866,0xd8a7c7daed6e7fd480d4cea3af336a97411fe25c,IntelButterfliesNRF19,blockv.nrf::ButterflyFolder::v2::Butterfly01::v2,None,229,2019-01-12 04:32:47+00:00,7051712,0x35f292d290ebd17f6b9a86cc3d92e81d8500c61d479e...
11019,0xfd89ea92f6ec07d955e2adbba2400ca1a6369028,SUPERWORLD,SuperWorld,None,667,2018-06-29 23:58:16+00:00,5877613,0x4015e3fd4e70db0006265458f3abe004a7bbabb74281...
11025,0x5536b6aadd29eaf0db112bb28046a5fad3761bd4,PXCN,PixelCons,None,705,2018-12-02 18:44:38+00:00,6814010,0x1e8f9420e7736444d6868d436a23efb022ee397c040b...
11052,0x909899c5dbb5002610dd8543b6f638be56e3b17e,PLASMABEARS,Plasma Bears,None,999,2019-03-12 23:45:38+00:00,7357333,0x8cd40be8345da2b99fc51574a709a9575690546a77e6...
11238,0xb55c5cac5014c662fdbf21a2c59cd45403c482fd,CLV,Clovers,None,3355,2019-08-16 20:38:38+00:00,8363701,0x6e5693d359d2816f4607864e0ce7b31a497248416278...
205824,0xd2c53d97273bbdd27fcb29adda965ed6ca7d1500,STACKX,StackX,18,1000000000000000000000000000000000000,2021-10-14 03:21:06+00:00,13413927,0xf0c248ca8664d33e6814a14dd38f2372a97b66ff4a47...
205870,0xf106a8a6118f5c70b6ef5c8925a623f45ec258d6,LP Token,FEGex PRO,18,1000000000000000000267300423504921343,2021-10-23 05:41:44+00:00,13471997,0x272d9f1670823c041b3f129b780ef7e2226a739fa291...
206042,0xb879d2de140452c94b573994b239b76d599b9853,bar,barter network,18,1000000000000000000000000000000000000000,2019-03-01 22:55:15+00:00,7286558,0x1ce59f2565cc770eedc30a049a8701ca7dce41dadfcd...


In [9]:
top10_address_by_count = (
    token_transfer_df
    [token_transfer_df.address.isin(token_df.address)]    
    .sort_values("totalCount",ascending=False)
    .iloc[:10]
    .address
)

In [10]:
token_df[token_df.address.isin(top10_address_by_count)]

,address,symbol,name,decimals,total_supply,block_timestamp,block_number,block_hash
60759,0xdac17f958d2ee523a2206206994597c13d831ec7,USDT,Tether USD,6,60057493363429,2017-11-28 00:41:21+00:00,4634748,0x1420761dbff0321c42a4bda163af2d7500d723b0139a...
68005,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,WBTC,Wrapped BTC,8,9387148386,2018-11-24 21:45:52+00:00,6766284,0x63baa5fa01a06cb5121871b235544d0eb7fa4e73a2d3...
92895,0x653430560be843c4a3d143d0110e896c2ab8ac0d,MOF,Molecular Future,16,1000000000000000000000000,2017-12-19 07:42:57+00:00,4758812,0x037da57c96152ce1ceab6dce4b3153717b86a757e97e...
121674,0x27c70cd1946795b66be9d954418546998b546634,LEASH,DOGE KILLER,18,122222220000000000000000,2020-09-01 22:07:48+00:00,10777939,0x1413f03aa1e38cdbba510344c035cbecfaaea5d61e6c...
127613,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,Wrapped Ether,18,2406609700709750988256543,2017-12-12 11:17:35+00:00,4719568,0xd6e5f60d6b2367e74cd2aa520dbeb104826c3932eb48...
178997,0x514910771af9ca656af840dff83e8264ecf986ca,LINK,ChainLink Token,18,1000000000000000000000000000,2017-09-16 21:26:29+00:00,4281611,0x44dd61bbf85faf839df366147928e35510957a29f1ce...
183070,0x4e15361fd6b4bb609fa63c81a2be19d873717870,FTM,Fantom Token,18,1987133655189934657006349375,2018-06-14 12:39:02+00:00,5787425,0x32536c97e0a194a6b1b158ee60a948f4b1e063b38387...
186018,0x0f5d2fb29fb7d3cfee444a200298f468908cc942,MANA,Decentraland MANA,18,2534650758473437434327976460,2017-08-15 19:18:09+00:00,4162059,0x827e4054b323d8b8ef63f731c5edc605102ec83cc625...
187900,0x9b9647431632af44be02ddd22477ed94d14aacaa,KOK,KOK Coin,18,5000000000000000000000000000,2020-02-24 03:33:43+00:00,9543746,0xe7115f852f072f01779cf8f8326ee039af079d080c82...
204655,0x95ad61b0a150d79219dcf64e1e6cc01f0b64c4ce,SHIB,SHIBA INU,18,1000000000000000000000000000000000,2020-07-31 18:32:43+00:00,10569013,0x678dc99c448fc2dbc10081160066b5f654c916340c79...


일단 여기서 가장 거래량이 많은 USDT와 WBTC를 한번 보자

## 2.  SWAP의 대상이 되는 토큰 살펴보기

### 2.1. UNISWAP V3의 SWAP Pool 찾기

Uniswap의 Pool을 탐색하기 위해서는 Uniswap V3의 Smart Contract를 이용해야 함

In [13]:
# USDT 토큰 주소와 WBTC 토큰 주소
# c.f) checksumAddress란, 잘못 오입력되지 않도록 체크섬 처리한 주소 정보
# EIP-55 스펙으로 정의됨 : https://github.com/ethereum/EIPs/blob/master/EIPS/eip-55.md#specification

USDT_ADDRESS = Web3.toChecksumAddress(
    "0xdac17f958d2ee523a2206206994597c13d831ec7"
)
WBTC_ADDRESS = Web3.toChecksumAddress(
    "0x2260fac5e5542a773aa44fbcfedf7c193bc2c599"
)

# UNISWAP ADDRESS 주소
UNISWAP_V3_FACTORY_ADDRESS = Web3.toChecksumAddress(
    "0x1F98431c8aD98523631AE4a59f267346ea31F984"
)

소스 코드를 실행하기 위해서는 스마트 컨트랙트의 Interface 정보가 필요하다. 이것은 소스코드를 컴파일하는 과정에서 ABI 파일 형태로 만들어지는데, etherscan같은 사이트에서는 주요 컨트랙트의 ABI 파일을 제공해준다.

In [14]:
@lru_cache(maxsize=1024)
def get_abi(contract_address:str)->str:
    # abi 가져오기
    apiKey = ETHERSCAN_DICT['apiKey']
    url = f"https://api.etherscan.io/api?apiKey={apiKey}&module=contract&action=getabi&address={contract_address}"
    return requests.get(url).json().get('result', "")

아래와 같이 구조 정보가 나와있다.

In [15]:
get_abi(UNISWAP_V3_FACTORY_ADDRESS)

'[{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"uint24","name":"fee","type":"uint24"},{"indexed":true,"internalType":"int24","name":"tickSpacing","type":"int24"}],"name":"FeeAmountEnabled","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"oldOwner","type":"address"},{"indexed":true,"internalType":"address","name":"newOwner","type":"address"}],"name":"OwnerChanged","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"token0","type":"address"},{"indexed":true,"internalType":"address","name":"token1","type":"address"},{"indexed":true,"internalType":"uint24","name":"fee","type":"uint24"},{"indexed":false,"internalType":"int24","name":"tickSpacing","type":"int24"},{"indexed":false,"internalType":"address","name":"pool","type":"address"}],"name":"PoolCreated","type":"event"},{"inputs":[{"internalType":"address","name":"tokenA","

컨트랙트와 컨트랙트 주소가 있으면, 컨트랙트에서 함수들을 호출할 수 있다. 
그런데 컨트랙트의 정보에서 붙기 위해서는 필수적으로 운영중인 메인넷의 이더리움 노드에 접근이 필수적이다. 자신의 로컬 컴퓨터를 메인넷의 이더리움 노드로 운영할 수도 있지만, infura와 같은 node provider을 이용할수도 잇다.

![](https://imgur.com/Se5U2nV.png)

In [18]:
def get_contract(contract_address):
    # infura 노드를 통해, Main넷에 붙기
    w3 = Web3(
        Web3.HTTPProvider(f"https://mainnet.infura.io/v3/{project_id}")
    )

    # Contract 가져오기
    return w3.eth.contract(
        address=Web3.toChecksumAddress(contract_address), 
        abi=get_abi(Web3.toChecksumAddress(contract_address)))

In [19]:
# 컨트랙트 정보 불러오기
contract = get_contract(UNISWAP_V3_FACTORY_ADDRESS)

In [23]:
# 컨트랙트에서 풀을 가져오는 방식 조회하기
func = contract.functions.getPool(Web3.toChecksumAddress(USDT_ADDRESS), 
                                  Web3.toChecksumAddress(WBTC_ADDRESS), 
                                  3000)

꼭 모든 함수가 Payable, 가스를 소요하는 것은 아니다. 가스를 소요하지 않는 함수들도 존재한다. 다만 블록체인 내 값의 변경을 요하는 건 payable 이어야 한다 (이더를 송금하거나, 토큰의 소유권을 바꾸는 등의 행위)

참고
````
Payable has nothing to do with it. You have to add either view or constant to make it a read-only function as the compiler will suggest you do if you remove payable because without value transfer, this function doesn't change anything. So, I think that's what happened, or something very similar.

This issue relates to Ethereum's treatment of state-changing (write) and read-only operations. In the case of a state-changing function the sender never gets the expected return value, but another contract does. The reason for this is the sent transactions have not been mined, so the return value is unknown.

The receipt you get is an identifier your node produced to let you know how to track the transaction. This callback confirms that your node is aware of your desire to submit something to the network. It's not the same concept as "result".

It's a confusing topic and it implies front-end design patterns that deal with waiting for mining. Have a look over here for an explainer: https://blog.b9lab.com/calls-vs-transactions-in-ethereum-smart-contracts-62d6b17d0bc2
````

In [24]:
# 풀을 조회하기
func.call()

'0x9Db9e0e53058C89e5B94e29621a205198648425B'

사실 이렇게 호출하는 방식이 etherscan에서 제공하는 대시보드 호출과도 동일하다. 

![](https://imgur.com/58BBeAv.png)

우리는 이제 `uniswap v3: WBTC-USDT`의 주소를 알게 되었다. => `0x9Db9e0e53058C89e5B94e29621a205198648425B`

여기서 SWAP이 발생되면 어떻게 될까? 이것은 모두 Event의 형태로 발행되며, event를 통해 우리는 어떤 트랜잭션이 서비스 내에서 발생했는지를 알수있게 된다.

이걸 검색하기 위해서는 bigquery 내에서 해당 address에서 발생되는 모든 event들을 보면 된다.

| column name | type  | mode      | description |
|  ---      | ---     | ---      | --- |
| log_index | INTEGER | REQUIRED | Integer of the log index position in the block | 
| transaction_hash | STRING | REQUIRED | Hash of the transactions this log was created from | 
| transaction_index | INTEGER | REQUIRED | Integer of the transactions index position log was created from | 
| address | STRING | NULLABLE | Address from which this log originated | 
| data | STRING | NULLABLE | Contains one or more 32 Bytes non-indexed arguments of the log | 
| topics | STRING | REPEATED | Indexed log arguments (0 to 4 32-byte hex strings). (In solidity: The first topic is the hash of the signature of the event (e.g. Deposit(address,bytes32,uint256)), except you declared the event with the anonymous specifier.) | 
| block_timestamp | TIMESTAMP | REQUIRED | Timestamp of the block where this log was in | 
| block_number | INTEGER | REQUIRED | The block number where this log was in | 
| block_hash | STRING | REQUIRED | Hash of the block where this log was in |

In [25]:
%%bigquery logs_df
SELECT address, transaction_hash, data, topics, block_timestamp
FROM `bigquery-public-data.crypto_ethereum.logs` 
WHERE 1=1
  AND lower(address) = '0x9db9e0e53058c89e5b94e29621a205198648425b'
  AND date(block_timestamp) >= '2021-11-10'


Downloading: 100%|██████████| 188/188 [00:04<00:00, 46.65rows/s]


In [28]:
get_abi("0x9db9e0e53058c89e5b94e29621a205198648425b")

'[{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"owner","type":"address"},{"indexed":true,"internalType":"int24","name":"tickLower","type":"int24"},{"indexed":true,"internalType":"int24","name":"tickUpper","type":"int24"},{"indexed":false,"internalType":"uint128","name":"amount","type":"uint128"},{"indexed":false,"internalType":"uint256","name":"amount0","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"amount1","type":"uint256"}],"name":"Burn","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"owner","type":"address"},{"indexed":false,"internalType":"address","name":"recipient","type":"address"},{"indexed":true,"internalType":"int24","name":"tickLower","type":"int24"},{"indexed":true,"internalType":"int24","name":"tickUpper","type":"int24"},{"indexed":false,"internalType":"uint128","name":"amount0","type":"uint128"},{"indexed":fa

In [39]:
row = logs_df.iloc[0]

이렇게 Bytes로 된 녀석들을 이제 파싱해야 한다. 이걸 처리하기 위해서 또 메인넷을 이용해서 디코딩이 가능하다. 

---

In [40]:
row

address                    0x9db9e0e53058c89e5b94e29621a205198648425b
transaction_hash    0x09b95059fd39c0943b9b17577d60141ce1101e455ff2...
data                0x00000000000000000000000000000000000000000000...
topics              [0xc42079f94a6350d7e6235f29174924f928cc2ac818e...
block_timestamp                             2021-11-10 18:46:08+00:00
Name: 0, dtype: object

In [41]:
abi = get_abi(row.address)
topic2abi = to_topic_map(abi)
event_abi = topic2abi[row.topics[0]]
event_name = event_abi['name']

In [45]:
json.loads(abi)[0]

{'inputs': [], 'stateMutability': 'nonpayable', 'type': 'constructor'}

In [49]:
for i, row in enumerate(json.loads(abi)):
    print(i, row)

0 {'inputs': [], 'stateMutability': 'nonpayable', 'type': 'constructor'}
1 {'anonymous': False, 'inputs': [{'indexed': True, 'internalType': 'address', 'name': 'owner', 'type': 'address'}, {'indexed': True, 'internalType': 'int24', 'name': 'tickLower', 'type': 'int24'}, {'indexed': True, 'internalType': 'int24', 'name': 'tickUpper', 'type': 'int24'}, {'indexed': False, 'internalType': 'uint128', 'name': 'amount', 'type': 'uint128'}, {'indexed': False, 'internalType': 'uint256', 'name': 'amount0', 'type': 'uint256'}, {'indexed': False, 'internalType': 'uint256', 'name': 'amount1', 'type': 'uint256'}], 'name': 'Burn', 'type': 'event'}
2 {'anonymous': False, 'inputs': [{'indexed': True, 'internalType': 'address', 'name': 'owner', 'type': 'address'}, {'indexed': False, 'internalType': 'address', 'name': 'recipient', 'type': 'address'}, {'indexed': True, 'internalType': 'int24', 'name': 'tickLower', 'type': 'int24'}, {'indexed': True, 'internalType': 'int24', 'name': 'tickUpper', 'type': 'i

In [42]:
topic2abi

{'0x0c396cd989a39f4459b5fa1aed6a9a8dcdbc45908acfd67e028cd568da98982c': {'anonymous': False,
  'inputs': [{'indexed': True,
    'internalType': 'address',
    'name': 'owner',
    'type': 'address'},
   {'indexed': True,
    'internalType': 'int24',
    'name': 'tickLower',
    'type': 'int24'},
   {'indexed': True,
    'internalType': 'int24',
    'name': 'tickUpper',
    'type': 'int24'},
   {'indexed': False,
    'internalType': 'uint128',
    'name': 'amount',
    'type': 'uint128'},
   {'indexed': False,
    'internalType': 'uint256',
    'name': 'amount0',
    'type': 'uint256'},
   {'indexed': False,
    'internalType': 'uint256',
    'name': 'amount1',
    'type': 'uint256'}],
  'name': 'Burn',
  'type': 'event'},
 '0x70935338e69775456a85ddef226c395fb668b63fa0115f5f20610b388e6ca9c0': {'anonymous': False,
  'inputs': [{'indexed': True,
    'internalType': 'address',
    'name': 'owner',
    'type': 'address'},
   {'indexed': False,
    'internalType': 'address',
    'name': 'reci

In [33]:
24 + 24 + 128 + 256 + 256

688

In [ ]:
row.data

In [ ]:
len(row.data[2:])

In [ ]:
len(row.data[2:])

In [ ]:
row.address

In [ ]:
row

In [ ]:
row.data

In [ ]:
(len(row.data[2:]) * 16 - 320)

In [ ]:
len(row.data)

In [ ]:
160 / 8

In [34]:
@lru_cache(maxsize=1024)
def to_topic_map(abi:str):
    from eth_utils import event_abi_to_log_topic
    
    topic_map = {}
    for row in json.loads(abi):
        if row['type'] == 'event':
            key = "0x" + event_abi_to_log_topic(row).hex()
            value = row
            topic_map[key] = value
    return topic_map


def decode_log(data, topics, contract_address):

    data = get_event_data(w3.codec, event_abi, log)['args']
    target_schema = event_abi['inputs']
    decoded_data = convert_to_hex(data, target_schema)


    return (evt_name, json.dumps(decoded_data), json.dumps(target_schema))

In [ ]:
target_data